In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
from itertools import combinations
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np

## Prep Data for training

In [39]:
conditions_diabetes = pd.read_csv('conditions_diabetes.csv')
# conditions_cancer = pd.read_csv('conditions_cancer.csv')
observations = pd.read_csv('observations_pivot.csv')
patients = pd.read_csv('patient_clean.csv')

In [40]:
patients

,PATIENT,birthdate,deathdate,ssn,drivers,passport,prefix,first,last,suffix,maiden,marital,race,ethnicity,gender,birthplace,address,curr_town
0,4ee2c837-e60f-4c54-9fdf-8686bc70760b,1929-04-08,2029-11-11,999-78-5976,NaN,NaN,NaN,Rosamaria,Pfannerstill,NaN,NaN,NaN,black,dominican,F,Pittsfield,18797 Karson Burgs Suite 444 Palmer Town MA 01...,Palmer
1,efaf74f9-3de3-45dd-a5d5-26d08e8a3190,2016-12-15,2020-02-19,999-59-9186,NaN,NaN,NaN,Loan,Bashirian,NaN,NaN,NaN,white,american,F,Medford,301 Eula Radial Suite 298 Brockton MA 02305 US,Brockton
2,aaa4c718-2f48-4c13-9ad0-d287cf280824,1943-11-28,2017-10-22,999-43-3780,S99992928,FALSE,Mr.,Angelo,Buckridge,NaN,NaN,S,black,african,M,Framingham,8693 Fred Crossroad New Bedford MA 02746 US,New
3,a1851c06-804e-4f31-9d8f-388cd52d4ad0,1954-10-22,2017-10-13,999-53-5542,S99975961,X98167138X,Mrs.,Cami,Terry,NaN,Schuster,M,white,english,F,Hudson,344 Olson Road Apt. 936 Attleboro MA 02703 US,Attleboro
4,48074b70-4db4-4ab0-b9e8-361bd2ba6216,1935-04-08,2017-09-06,999-34-8549,S99997003,X65866752X,Mr.,Giovanni,Russel,NaN,NaN,M,hispanic,puerto_rican,M,Westfield,5780 Corwin Trafficway Dartmouth MA 02714 US,Dartmouth
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,1131717c-e009-446d-b353-0b865cbb4194,1932-01-20,NaN,999-45-9346,S99967647,X18790808X,Mrs.,Seema,McGlynn,NaN,Hilll,M,black,dominican,F,Springfield,4315 Sawayn Port Greenfield Town MA 01302 US,Greenfield
1458,462abede-dc9a-4928-a120-0449155d6ada,1931-10-11,NaN,999-70-4419,S99951366,FALSE,Mrs.,Elenora,Ankunding,NaN,Morissette,M,white,polish,F,Boston,2342 River Oval Suite 439 Berlin MA 01503 US,NaN
1459,6e687318-9491-4453-8877-da816d835499,1931-05-22,NaN,999-28-2963,S99954452,FALSE,Mr.,Raymundo,Rau,NaN,NaN,M,white,polish,M,Mashpee,20031 Bailey Keys Apt. 161 Boston MA 02284 US,Boston
1460,c0cb1ff4-8fbe-4f93-990d-abf62284cb4d,1931-04-20,NaN,999-18-1247,S99929550,FALSE,Mrs.,Chan,Schaden,NaN,Erdman,M,white,irish,F,Marblehead,1218 Heaney Wall East Longmeadow MA 01028 US,East


In [41]:
le = LabelEncoder()

def prep_data(patients, conditions, illness_descriptions, observations):
    patients.rename(columns={'patient':'PATIENT'}, inplace=True)
    patients = patients.drop(columns=['birthdate', 'marital','deathdate', 'address','ssn', 'drivers', 'passport', 'prefix', 'first', 'last', 'suffix', 'maiden'])
    
    patients = patients.dropna()
    conditions = conditions.dropna()

    # MERGE DATASETS
    merged_df = pd.merge(patients, conditions, on='PATIENT', how='left')
    merged_df = pd.merge(merged_df, observations, on='PATIENT', how='left')

    merged_df["y"] = (merged_df[illness_descriptions] == 1).any(axis=1).astype(int)
    
    merged_df = merged_df.drop(columns=illness_descriptions)
    merged_df["race"] = le.fit_transform(merged_df["race"]) 
    race_code = {code: race for code, race in enumerate(le.classes_)}


    merged_df["ethnicity"] = le.fit_transform(merged_df["ethnicity"])
    eth_code = {code: ethnicity for code, ethnicity in enumerate(le.classes_)}

    merged_df["gender"] = le.fit_transform(merged_df["gender"])  
    gen_code = {code: gender for code, gender in enumerate(le.classes_)}

    merged_df["birthplace"] = le.fit_transform(merged_df["birthplace"]) 
    bp_code = {code: bp for code, bp in enumerate(le.classes_)}

    merged_df["curr_town"] = le.fit_transform(merged_df["birthplace"]) 
    curr_code = {code: bp for code, bp in enumerate(le.classes_)}


    # split into test and train
    train, test = train_test_split(merged_df, test_size=0.2, random_state=42)
    
    # Y column to predict is diabetes
    X_train = train.drop(columns=['y'])
    y_train = train['y']
    
    X_test = test.drop(columns=['y'])
    y_test = test['y']
    
    return X_train, y_train, X_test, y_test, race_code, eth_code, gen_code, bp_code

illness_descriptions = ['PATIENT','Diabetes_CONDITIONS','Prediabetes_CONDITIONS','Diabetic retinopathy associated with type II diabetes mellitus (disorder)_CONDITIONS', 
                        'Nonproliferative diabetic retinopathy due to type 2 diabetes mellitus (disorder)_CONDITIONS', 'Macular edema and retinopathy due to type 2 diabetes mellitus (disorder)_CONDITIONS', 
                        'Microalbuminuria due to type 2 diabetes mellitus (disorder)_CONDITIONS', 'Diabetic renal disease (disorder)_CONDITIONS', 'Neuropathy due to type 2 diabetes mellitus (disorder)_CONDITIONS']
X_train, y_train, X_test, y_test, race_code, eth_code, gen_code, bp_code = prep_data(patients, conditions_diabetes, illness_descriptions, observations)

In [42]:
#LogisticRegression
LR = LogisticRegression(max_iter=10000000000000000000)
LRScore = cross_val_score(LR, X_train, y_train, cv=5).mean()

# keep track of best Logistic Regression Score

#DecisionTreeClassifier
param_grid = { 'max_depth': [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, None ]}

tree = DecisionTreeClassifier()
grid_search = GridSearchCV(tree, param_grid, cv=5)
grid_search.fit(X_train, y_train)
DTCScore  = grid_search.best_score_
bestDTCDepth = grid_search.best_params_


# Random Forrest Classifier    
forrest = RandomForestClassifier(random_state=0)
grid_search = GridSearchCV(forrest, param_grid, cv=5)
grid_search.fit(X_train, y_train)

RFCScore  = grid_search.best_score_
bestRFCDepth = grid_search.best_params_

#SVC
SVM = SVC()

# use grid search to find best gamma for SVM
g = {'gamma': 10.0 ** np.arange(-5, 5) }
grid_search = GridSearchCV(SVM, g, cv=5)
grid_search.fit(X_train, y_train)

SVMScore  = grid_search.best_score_   


print("best LR :", LRScore)
print("best DTC:", DTCScore)
print("best max depth: ", bestDTCDepth)
print("best RFC: ", RFCScore)
print("best max depth: ", bestRFCDepth)
print("best SVM: ", SVMScore)

max_score = 0
max_model = ""
if LRScore > max_score:
    max_score = LRScore
    max_model = "LR"
if DTCScore > max_score:
    max_score = DTCScore
    max_model = "DTC"
if RFCScore > max_score:
    max_score = RFCScore
    max_model = "RFC"
if SVMScore > max_score:
    max_score = SVMScore
    max_model = "SVM"

print("best score overall is: ", max_score, " with model: ", max_model)

KeyboardInterrupt: 

Next would compute risk scores!

In [ ]:
race_code

{0: 'asian', 1: 'black', 2: 'hispanic', 3: 'white'}

Predict probabilities for all our entries using the best model we found

In [ ]:
forrest = RandomForestClassifier(max_depth=5)
forrest.fit(X_train, y_train)
pred_prob = forrest.predict_proba(X_test)

Define average risk score finding function

In [ ]:
def find_risk(code, col, probs):
    indices = (X_test[col] == code)
    prob_subset = probs[indices]
    av_prob = np.mean(prob_subset[:, 1]) 
    return av_prob   

Compute av. risk score for Asian patients

In [ ]:
find_risk(0, 'race', pred_prob)

0.4798957529442787

Compute av. risk score for Black patients

In [ ]:
find_risk(1, 'race', pred_prob)

0.23560657020876027

Compute av. risk score for Hispanic patients

In [ ]:
find_risk(2, 'race', pred_prob)

0.3262452568831123

Compute av. risk score for white patients

In [ ]:
find_risk(3, 'race', pred_prob)

0.3158189002160418

Compute av. risk for women

In [ ]:
gen_code

{0: 'F', 1: 'M'}

In [ ]:
find_risk(0, 'gender', pred_prob)

0.36914065574253196

In [ ]:
find_risk(1, 'gender', pred_prob)

0.2668203329823863

ethnicity

In [ ]:
av_risk_eth = []

for code, name in eth_code.items():
    av = find_risk(code, 'ethnicity', pred_prob)
    new_row = {'eth': name, 'risk': av}
    av_risk_eth.append(new_row)

av_risk_eth_df = pd.DataFrame(av_risk_eth)
av_risk_eth_df = av_risk_eth_df.sort_values(by='risk', ascending=False)


In [ ]:
av_risk_eth_df

,eth,risk
2,asian_indian,0.713611
13,polish,0.577234
9,german,0.496564
12,mexican,0.431718
1,american,0.424515
14,portuguese,0.396477
6,english,0.371829
17,scottish,0.333333
11,italian,0.318030
15,puerto_rican,0.310162


keep taking people from the top and bottom until its 20 on each side, then find av risk score

In [ ]:
richTowns = ["Dover", "Weston", "Wellesley", "Lexington", "Sherborn", "Cohasset", "Lincoln", "Carlisle", "Hingham", "Winchester", 
                "Medfield", "Concord", "Needham", "Sudbury", "Hopkinton", "Boxford", "Brookline", "Andover",  
                  "Southborough", "Belmont", "Acton", "Marblehead", "Newton", "Nantucket", "Duxbury", "Boxborough", "Westwood","Natick", 
                  "Longmeadow", "Marion", "Groton", "Newbury", "North Andover", "Sharon", "Arlington", "Norwell", "Reading", 
                  "Lynnfield", "Marshfield", "Holliston", "Medway", "Canton", "Milton", "Ipswich", "Littleton", "Westford", "North Reading", "Chelmsford", "Dedham",
                  "Walpole", "Mansfield", "Shrewsbury", "Norwood", "Hanover", "Stow", "Newburyport", "Chatham", "Orleans", "Harwich",
                  "Swampscott","Fairhaven", "Salem"]

poorTowns = ["Springfield", "Lawrence", "Holyoke", "Amherst", "New Bedford", "Chelsea", "Fall River", "Athol", "Orange", "Lynn", "Fitchburg", "Gardner", "Brockton", "Malden", "Worcester", "Chicopee", "North Adams", "Everett",
    "Ware", "Dudley", "Greenfield Town", "Weymouth Town", "Montague", "Revere", "Taunton", "Adams", "Huntington", "Charlemont", "Leominster", "Florida", "Colrain", "Hardwick",
    "Palmer Town", "Peabody", "Somerville", "Lowell", "Westfield", "Billerica"]

Having trouble appending town names to dataframe with town codes and people counts

need that so that we can then get the top twenty of each 
to then average to get rich likelihood and poor likelihood of diabetes 
yayyyy 

Create a df with all the information for teh rich and poor towns

In [ ]:
def find_town_info(town, bp_code_swapped, townCounts_df):
    code = bp_code_swapped[town]
    
    if not townCounts_df[townCounts_df['birthplace'] == code].empty:
        count = townCounts_df[townCounts_df['birthplace'] == code]['count'].values[0]
    else:
        count = 0
    
    new_row = {'birthplace': town, 'code': code, 'count': count}
    
    new_row_df = pd.DataFrame([new_row])
    
    return new_row_df

In [ ]:
birthplace_counts = X_test.groupby('birthplace').size().reset_index(name='count')

townCounts_df = pd.merge(X_test, birthplace_counts, on='birthplace')
town_info_rich = pd.DataFrame(columns=['birthplace', 'code', 'count'])
town_info_poor = pd.DataFrame(columns=['birthplace', 'code', 'count'])

bp_code_swapped = {value: key for key, value in bp_code.items()}

for town in richTowns:
    
    new_row_df = find_town_info(town, bp_code_swapped, townCounts_df)
    town_info_rich = pd.concat([town_info_rich, new_row_df], ignore_index=True)

for town in poorTowns:
    
    new_row_df = find_town_info(town, bp_code_swapped, townCounts_df)
    town_info_poor= pd.concat([town_info_poor, new_row_df], ignore_index=True)


## proceed with the following part to get top 65 people from each rich and poor 

In [ ]:
def get_towns_by_sum_pop(town_info, col):
    
    townsUsed = set()
    peopleCount = 0

    for index, row in town_info.iterrows():
        
        if peopleCount > 65:
            break
        
        birthplace = row[col]
        count = row['count']
        townsUsed.add(birthplace)
        peopleCount += count
    
    return townsUsed, peopleCount

richTownsUsed, richPeopleCount = get_towns_by_sum_pop(town_info_rich, 'birthplace')
poorTownsUsed, poorPeopleCount = get_towns_by_sum_pop(town_info_poor, 'birthplace')

In [ ]:
rich_town_codes = []

for town_full in richTownsUsed:
    rich_town_codes.append(bp_code_swapped[town_full])

In [ ]:
indices = X_test['birthplace'].isin(rich_town_codes)
prob_subset = pred_prob[indices]
av_rich_prob = np.mean(prob_subset[:, 1]) 

av_rich_prob

0.33186990727615645

In [ ]:
poor_town_codes = []

for town_full in poorTownsUsed:
    poor_town_codes.append(bp_code_swapped[town_full])

In [ ]:
indices = X_test['birthplace'].isin(poor_town_codes)
prob_subset = pred_prob[indices]
av_poor_prob = np.mean(prob_subset[:, 1]) 

av_poor_prob

0.32202650721201703

This is a really annoying result...

## Repeating Process for Current Town of Residence

In [ ]:
richTownsUsed, richPeopleCount = get_towns_by_sum_pop(town_info_rich, 'curr_town')
poorTownsUsed, poorPeopleCount = get_towns_by_sum_pop(town_info_poor, 'curr_town')

KeyError: 'curr_town'